In [33]:
import deeprefine
import torch
import pdbfixer
from openmm.app import PDBFile

In [24]:
!wget https://files.rcsb.org/download/1DUR.pdb -q -O 1DUR.pdb

In [34]:
fixer = pdbfixer.PDBFixer(filename='1DUR.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()

PDBFile.writeFile(fixer.topology, fixer.positions, open('1DUR_fixed.pdb', 'w'))


In [36]:
import mdtraj

traj = mdtraj.load('chignolin_ensemble.pdb')
top = traj.topology
traj

/n/hekstra_lab/people/ziyuan/.conda/envs/jax/lib/python3.10/site-packages/mdtraj/formats/pdb/pdbfile.py:200: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '


<mdtraj.Trajectory with 18 frames, 138 atoms, 10 residues, without unitcells at 0x2b3ab9986920>

In [37]:
converter = deeprefine.ICConverter(top, vec_angles=False)

In [49]:
xyz0 = torch.Tensor(traj.xyz.reshape(18, -1))


In [50]:
xyz0 = torch.tile(xyz0, (10000, 1))

In [51]:
xyz0.shape

torch.Size([180000, 414])

In [52]:
%%timeit
converter.xyz2ic(xyz0).reshape(18, -1, 3)

7.12 s ± 70.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
ic = converter.xyz2ic(xyz0)

In [43]:
ic.reshape(18, -1, 3).shape

torch.Size([18, 138, 3])

In [45]:
ic.reshape(18, -1, 3)[0, 29:]

tensor([[-1.8770e-01, -5.0310e-01,  5.0140e-01],
        [ 1.0389e-01,  1.9100e+00,  3.7659e-01],
        [ 1.2150e-01,  2.1169e+00,  5.7109e-02],
        [ 1.0408e-01,  1.9093e+00, -1.7187e+00],
        [ 1.0389e-01,  1.9105e+00,  2.4697e+00],
        [ 9.7916e-02,  2.0944e+00,  1.6627e+00],
        [ 1.2146e-01,  2.1170e+00, -1.4375e+00],
        [ 9.8008e-02,  2.0962e+00,  1.8859e+00],
        [ 1.2151e-01,  2.0976e+00, -1.1295e+00],
        [ 1.2149e-01,  2.1167e+00,  2.9094e+00],
        [ 9.7972e-02,  2.0960e+00,  2.0806e+00],
        [ 1.2150e-01,  2.1195e+00,  2.4463e+00],
        [ 9.7990e-02,  2.0930e+00,  1.2670e+00],
        [ 1.2143e-01,  2.1186e+00,  2.7424e+00],
        [ 9.8064e-02,  2.0933e+00, -1.5631e+00],
        [ 1.2159e-01,  2.1138e+00, -2.5478e+00],
        [ 9.7982e-02,  2.0978e+00,  3.3615e-01],
        [ 1.2149e-01,  2.1183e+00, -2.5881e-01],
        [ 9.7623e-02,  2.0895e+00,  1.9337e+00],
        [ 1.2149e-01,  2.1155e+00, -1.2726e+00],
        [ 9.7988e-02

In [19]:
%%timeit
converter.ic2xyz(ic).reshape(18, -1, 3)

1.8 ms ± 5.82 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
torch.max(torch.abs(xyz_new - xyz0))

tensor(0.1796)

In [23]:
torch.mean(torch.sqrt(torch.sum((xyz0-xyz_new)**2, dim=-1)))

tensor(0.2468)